### You must clone the [opnbnchmark repo](https://github.com/opnbnch/opnbnchmark) and append the path to access the methods in the repository.

If you have any questions, please drop them in the [Issues](https://github.com/opnbnch/opnbnchmark/issues) for the opnbnch Github repo or reference the parent newsletter post.

In [ ]:
import os
import sys

OPNBNCHMRK_HOME = '../'

sys.path.append(OPNBNCHMRK_HOME)

import pandas as pd
import rdkit

from produce_meta import produce_meta
from standardize import standardize
from resolve_class import resolve_class
from utils.meta_utils import read_meta

#### First, start with a visual inspection of the data

In [ ]:
data_dir = OPNBNCHMRK_HOME + 'case_studies/Martins_et_al_2012/'
data_file = 'martins_et_al_2012.csv'
data_path = os.path.join(data_dir, data_file)


pd.read_csv(data_path).head()

What do each of the four columns in the dataset contain?

* **num:** a redundant index column

* **name:** a column of compound names and ids. Also appears to have limited utility. 

* **p_np:** a column that appears to hold a class encoding (p = penetrative vs. np = non_penetrative)

* **smiles:** a column specifying the structure for each compound
 
The two columns holding the relevant data for our purposes are **p_np** and **smiles.** We know they are what we need to extract, but before we can start hacking away at this dataset, it's important that we set up a system to track our curation decisions, starting with the raw literature source.


### `produce_meta.py`

The `produce_meta` method from `produce_meta.py` assists in doing this. By providing a DOI (only ACS supported at the moment) and a path to the dataset we want to clean, we can produce an initial `metadata.json` file that will track progress through the data wrangling process. We believe that assiduously tracking the steps taken in data curation is crucial for assessing the provenance of benchmark data and ensuring its reproducibility. Thus, we begin: 

In [ ]:
produce_meta('https://doi.org/10.1021/ci300124c', data_path)

And so, our metadata.json file is produced. In it we store the title, author, and other article metadata, as well as some  basic info about the dataset being targeted for curation. We can take a look at our starting metadata using the `cat` command 

In [ ]:
! cat ../case_studies/Martins_et_al_2012/Martins_et_al_2012_metadata.json

Now we’re ready to start making changes to the raw data file from Martins et al. The first order of business is to standardize SMILES strings and class labels.

### `standardize.py` 

For this we use the `standardize` method from `standardize.py.` All we need to supply to the method is a path to the directory where our raw data resides, the name of the column which contains the SMILES data, and the name of the column containing class labels. From our visual analysis of the data above, those two columns are `smiles` and `p_np` respectively.

Non-standard class labels are mapped into integres 0 to n for n unique classes. The standardize function will ask for manual input to achieve a mapping from custom class labels {n, p} to standardized class labels {0, 1}. In this case, n → 0 and p → 1. 

In [ ]:
standardize(data_dir, smiles_col='smiles', class_col='p_np')

Once we enter that mapping, we’ll see that it is included in the metadata file alongside information on invalid SMILES strings. Such extensive documentation is included in the `metadata.json` to create a historical record of the decisions made during the curation process and to encourage reproducibility. Thankfully, this metadata is produced in an automated fashion and will be stored right alongside our standardized data.

In [ ]:
! cat ../case_studies/Martins_et_al_2012/Martins_et_al_2012_metadata.json

### Invalid SMILES

I'm always interested in taking a closer look at invalid SMILES, just to make sure we didn't unncessarily drop any valuable data! 

In [ ]:
meta = read_meta(data_dir)
raw_data_path = meta.get('data_path')
std_data_path = meta.get('std_data_path')
std_smiles_col = meta.get('std_smiles_col')
smiles_col = meta.get('smiles_col')

invalid_smiles = pd.read_csv(std_data_path) \
    .loc[lambda x:x[std_smiles_col] == 'invalid_smiles'] \
    .loc[::, smiles_col] \
    .to_list()

pd.read_csv(std_data_path) \
    .loc[lambda x:x[smiles_col].isin(invalid_smiles)]

Alas, it looks like none of our defective SMILES were given a `name` that can be resolved to a proper structure, and they indeed appear to be invalid. One can read the error logging from rdkit produced by `standardize` to get a sense for why SMILES fail. In this case, many SMILES fail due to attempts at specifying impossible nitrogen chemistry, e.g.:

    RDKit ERROR: Explicit valence for atom # 5 N, 4, is greater than permitted
    
### Replicate Entries

With SMILES standardized, it’s easy to see that there are certain data points for which multiple class records are included in Martins et al.’s dataset. For example, below is a list of SMILES that appear multiple times in the dataset and their associated counts.

In [ ]:
pd.read_csv(std_data_path) \
    .loc[lambda x:x[std_smiles_col] != 'invalid_smiles'] \
    .loc[::, std_smiles_col] \
    .value_counts().head(10)

We must resolve each of these replicate groupings to a single class, so that we can train our model on a functional mapping between SMILES structures and BBB penetration labels. 

### resolve_class.py

For this purpose, we’ve built the `resolve_class` method, which takes in the path to the data directory and an optional “filter function” parameter. 

In [ ]:
resolve_class(data_dir, filter_fn=None)

In this example, we don’t specify a filter function, so the method prompts us for one at the command line.  Currently, two filtering methods are supported to handle class conflicts within a replicate group. For `filter_fn=”unanimous”`, all replicates must share the same class, otherwise they are removed from the training data. For `filter_fn=”majority”`, a simple majority suffices. For instance, if two replicates claim to be penetrative (p) and one non-penetrative (n), then a single replicate with label p is chosen to represent the group under the majority filter.

The resolve_class method appends metadata that includes the mapping from every replicate group’s InChI key into the row index selected to represent that group in the filtered data. Also appended is metadata on the number of remaining rows and the function used to filter the conflicting replicates.

In [ ]:
!cat ../case_studies/Martins_et_al_2012/Martins_et_al_2012_metadata.json

With SMILES standardized and classes resolved, our final step is to select those columns we want to keep for training purposes. From our original 2053 rows, we end up with 1951 unique SMILES, each mapped into a resolved standard class. This data is now squeaky clean and fit for modeling. 

In [ ]:
meta = read_meta(data_dir)
resolved_data_path = meta.get('resolved_data_path')
train = pd.read_csv(resolved_data_path)[['std_smiles', 'std_class']]
train

## Conclusion

This demonstration just begins to scratch the surface of what we hope to create with the [OpenBenchmark](https://github.com/opnbnch/opnbnchmrk) project. We plan to continue constructing and curating reliable, open ADME-Tox Benchmarks, with new benchmarks landing every two weeks. To keep abreast of developments on the OpenBenchmark project, please [subscribe to the newsletter.](https://opnbnch.substack.com/welcome)

To run your own compounds through OpenBench’s state-of-the-art BBB classifier at the click-of-a-button, sign up for the [OpenBench Lab](https://lab.opnbnch.com) today. It’s free and simple to use.